In [59]:
import bs4
from urllib.request import urlopen
import csv
import dateutil.parser as dparser

In [32]:
def readUrl(url):
    web_data = urlopen(url)
    html = web_data.read()
    web_data.close()
    data = bs4.BeautifulSoup(html, 'html.parser')
    return data

In [33]:
url = "https://moveek.com/phim-viet-nam/" 
soup = readUrl(url) 
results = soup.findAll('div','panel panel-post panel-movie-grid') 
print(results)

[<div class="panel panel-post panel-movie-grid">
<a href="/phim/cam-bay-hoi-tho-cua-quy/" title="Cạm Bẫy: Hơi Thở Của Quỷ">
<div class="panel-rating-box">
                                                                                            12/01
                                            
                                                                                            <img src="https://i.moveek.com/bundles/ornweb/img/icon-good.png" width="20"/>
                                                80%
                                                                                    </div>
<img alt="Cạm Bẫy: Hơi Thở Của Quỷ" class="lazyload img-responsive post-thumbnail" data-src="https://i.moveek.com/media/cache/poster_small/media/2f0abfe37458251f24ff5614718da7485a5b2dc5.jpg" data-srcset="https://i.moveek.com/media/cache/poster_small/media/2f0abfe37458251f24ff5614718da7485a5b2dc5.jpg 1x, https://i.moveek.com/media/cache/poster_fixed/media/2f0abfe37458251f24ff5614718da748

In [34]:
#Get Name
def getName(link):
    soup = readUrl(link)
    resultFilms = soup.find('div','movie-header-wrapper') 
    srcName = resultFilms.find('img')
    name = srcName.get('alt')
    return name;

#Get percent 
def getPercent(link):
    soup2 = readUrl(link)
    resultFilms = soup2.find('div','movie-header-wrapper')
    #Get percents
    percent = resultFilms.find('div','action chart-pie chart-pie-small pull-left') 
    return percent.getText();

#Get Director 
def getDirector(link):
    soup3 = readUrl(link)
    resultFilms = soup3.find('div','movie-header-wrapper') 
    srcDirector = resultFilms.findAll('p','text-white')
    director = srcDirector[len(srcDirector)-1].find('a').getText();
    return director

#Get Description
def getDescription(link):
    soup = readUrl(link)
    resultDescription = soup.find('div','synopsis m-t m-b-xs')
    description = resultDescription.getText()
    return description
    
#Get actor
def getActor(link):
    soup = readUrl(link)
    resultActorFilms = soup.find('div','movie-actors panel wrapper m-t')
    actors = resultActorFilms.findAll('img')
    data = []
    for actor in actors:
        data.append(actor.get('alt'))
    return data

#Get time
def getTime(link) :
    soup = readUrl(link)
    resultTime = soup.find('div','movie-header-wrapper')
    srcTime = resultTime.find('a','action action-imdb btn btn-rounded btn-lg btn-icon btn-default')
    time = srcTime.getText()
    return time
#example
link2 = 'https://moveek.com/phim/cam-bay-hoi-tho-cua-quy/'
name = getName(link2)
percent = getPercent(link2)
director = getDirector(link2)
description = getDescription(link2)
actors = getActor(link2)
time = getTime(link2)
# print(name)
# print(percent)
print(director)
# print(description)
# print(time)

# actors

Nguyễn Quang Tuyến


In [67]:
rows = [];
rows.append(['Film', 'Imdb','Director','Description','Star', 'Time', 'Poster', 'Date'])
for result in results:
    data = result.find('a')
    
    #Get link film
    link = 'https://moveek.com'+data.get('href')
    #Get data from link
    soup = readUrl(link)
    resultFilms = soup.find('div','movie-header-wrapper')
    
    #1- Get Namme Film
    srcName = resultFilms.find('img')
    name = srcName.get('alt')
    
    #2 get poster
    srcPoster = resultFilms.find('img')
    poster = srcPoster.get('data-srcset')
    if poster == None:
        linkPoster = ''
    else:
        poster = poster.split(", ")
        linkPoster = poster[1].replace("2x","")
    
    #2 - Get percents
    percent = resultFilms.find('div','action chart-pie chart-pie-small pull-left')
    if(percent == None) :
        percent = '5'
    else :
        percent = percent.getText().replace("%","")
        percent = int(percent)/10
    print(percent)
        
    # get date
    srcDate = resultFilms.findAll('p','text-white')
    date = srcDate[-1].getText().split("\n")[1].strip()
    date = dparser.parse(date,fuzzy=True)
    date = str(date.date())
    
        
    #3 - Get director
    srcDirector = resultFilms.findAll('p','text-white')
    director = srcDirector[len(srcDirector)-1].find('a');
    if(director == None):
        director = ''
    else:
        director = director.getText()
    
    #4 - Get description
    description = soup.find('div','synopsis m-t m-b-xs')
    if(description == None):
        description=''
    else :
        description = description.getText()

    #5 Get Time
    time = resultFilms.find('a','action action-imdb btn btn-rounded btn-lg btn-icon btn-default')
    if(time == None):
        time = '100phút'
    else :
        time = time.getText()
        if "ph" not in time:
            time = '100phút'
    
    #6 - Get Actors
    resultActorFilms = soup.find('div','movie-actors panel wrapper m-t')
    listActor=[];
    if(resultActorFilms == None) :
        listActor= 'No actor'
    else :
        actors = resultActorFilms.findAll('img')
        for actor in actors:
            listActor.append(actor.get('alt'))
    rows.append([name,percent, director,''+description.strip(),listActor,time, linkPoster, date])

8.0
7.9
5.0
4.0
5
9.1
8.8
8.2
6.0
2.9
9.3
5
8.2
4.7
5
8.1
8.1
7.4
5.0
8.7
6.8
8.3
2.5
3.6
7.3
10.0
9.5
9.2
9.3
9.1
1.0
7.7
5
6.7
7.1
8.0
5
5
5
5
5
5
5


In [68]:
# Create csv and write rows to output file
# print(rows)
with open('FullInfoFilmTrain.csv','w', newline='',encoding="utf-8-sig") as f_output:
    csv_output = csv.writer(f_output)
    csv_output.writerows(rows)
print('Success')

Success


In [9]:
#Kien
rows = [];
for result in results:
    data = result.find('a')
    #Get link film
    link = 'https://moveek.com'+data.get('href')
    #Get data from link
    soup = readUrl(link)
    resultFilms = soup.find('div','movie-header-wrapper')
    print(link)
    #3 - Get director
    srcDirector = resultFilms.findAll('p','text-white')
    director = srcDirector[len(srcDirector)-1].find('a');
    if(director == None):
        director = ''
    else:
        director = director.getText()
    rows.append(director)


https://moveek.com/phim/cam-bay-hoi-tho-cua-quy/
https://moveek.com/phim/o-day-co-nang/
https://moveek.com/phim/xuong-13/
https://moveek.com/phim/lala-hay-de-em-yeu-anh/
https://moveek.com/phim/the-vietnamese-bride/
https://moveek.com/phim/sieu-sao-sieu-ngo/
https://moveek.com/phim/ve-que-an-tet/
https://moveek.com/phim/798-muoi/
https://moveek.com/phim/dich-ton-doc-dac/
https://moveek.com/phim/thu-yeu-roi-biet/
https://moveek.com/phim/thang-nam-ruc-ro/
https://moveek.com/phim/phan-van-khai-nguoi-con-dat-thep/
https://moveek.com/phim/ong-ngoai-tuoi-30/
https://moveek.com/phim/yeu-em-bat-chap/
https://moveek.com/phim/ha-cuoi-tinh-dau/
https://moveek.com/phim/lat-mat-3-ba-chang-khuyet/
https://moveek.com/phim/100-ngay-ben-em/
https://moveek.com/phim/11-niem-hy-vong/
https://moveek.com/phim/yeu-nu-sieu-quay/
https://moveek.com/phim/nham-mat-thay-mua-he/
https://moveek.com/phim/em-gai-mua/
https://moveek.com/phim/ong-kinh-sat-nhan/
https://moveek.com/phim/lo-mat/
https://moveek.com/phim/tr

In [17]:
#Kien

# print(rows)
# with open('KienTrain.csv','w', newline='',encoding="utf-8-sig") as f_output:
#     csv_output = csv.writer(f_output)
#     csv_output.writerows(rows)
# print('Success')
result = []
for row in rows:
    if len(row) is not 0:
        split_name = row.split()
        for name in split_name:
            result.append(name)
print(result)

['Nguyễn', 'Quang', 'Tuyến', 'Đỗ', 'Nam', 'Phan', 'Minh', 'Han', 'Sang-hee', 'Tsai', 'Lex', 'Đức', 'Thịnh', 'Nguyễn', 'Hoàng', 'Anh', 'Dustin', 'Nguyễn', 'Trần', 'Ngọc', 'Giàu', 'Nguyễn', 'Hà', 'Nguyễn', 'Quang', 'Dũng', 'Lâm', 'Tư', 'Võ', 'Thanh', 'Hòa', 'Văn', 'Công', 'Viễn', 'Trương', 'Quang', 'Thịnh', 'Lý', 'Hải', 'Vũ', 'Ngọc', 'Phượng', 'Robie', 'Trường', 'Nguyễn', 'Ngọc', 'Hùng', 'Cao', 'Thúy', 'Nhi', 'Kawaii', 'Nguyễn', 'Tuấn', 'Anh', 'Hứa', 'Vĩ', 'Văn', 'Vĩnh', 'Thuyên', 'Duy', 'Joseph', 'Lương', 'Trung', 'Tín', 'Leon', 'Quang', 'Lê', 'Charlie', 'Nguyễn', 'Thắng', 'Vũ', 'Võ', 'Thanh', 'Hòa', 'Nguyễn', 'Thành', 'Vinh', 'Đinh', 'Tuấn', 'Vũ', 'Trần', 'Nhân', 'Kiên', 'Swann', 'Dubus', 'Nguyễn', 'Hoàng', 'Duy', 'Victor', 'Vũ', 'Roland', 'Nguyễn', 'Nhân', 'Nguyễn', 'Đức', 'Minh', 'Cường', 'Ngô', 'Bảo', 'Nhân', 'Ken', 'Ochiai', 'Hàm', 'Trần']
